In [0]:
%run "./bronzecommonutilities"


In [0]:
dbutils.widgets.text("modulename","sales")

In [0]:
modulename = dbutils.widgets.get("modulename")
config = load_config(f"/Volumes/bronze/da_sales_dev/salesvolume/bronzeconfig{modulename}.json")

for tables in config["tables"]:
    table_name = tables["name"]
    source_path = tables["path"]
    checkpoint_path = f"/Volumes/bronze/da_sales_dev/{modulename}volume/tmp/schema/{table_name}"
    if tables["loadtype"] == 'append':
        df = (spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")           # The format of the files landing in the volume
        .option("cloudFiles.schemaLocation", checkpoint_path) # Stores inferred schema
        .option("header", "true") 
        .option("ignoreMissingFiles", "true")                   # Specific to CSV
        .load(f"{source_path}"))
        (df.writeStream
        .option("checkpointLocation", checkpoint_path)
        .trigger(availableNow=True)                   # Good for testing: processes all current files and stops
        .toTable(f"bronze.da_{modulename}_dev.{table_name}"))
    else:
        df = spark.read.format(f"{tables["filetype"]}").option("header", "true").load(f"{source_path}")
        #(df.write.mode("overwrite").saveAsTable(f"bronze.da_{modulename}_dev.{table_name}"))
        df.createOrReplaceTempView("temp_data_to_overwrite")
        spark.sql(f"""
            CREATE OR REPLACE TABLE bronze.da_{modulename}_dev.{table_name}
            AS SELECT * FROM temp_data_to_overwrite
        """)
